## Content
1. [Data loading](#1.-Data-loading)  
2. [Structure of the data](#2.-Structure-of-the-data)  
3. [Data preprocessing](#3.-Data-preprocessing)  
    3.1 [Context/passage sentence tokenization](#3.1-Context/passage-sentence-tokenization-[1])  
    3.2 [Word tokenization](#3.2-Word-tokenization-[1])  
    3.3 [Lemmatization](#3.3-Lemmatization-[2])  
    3.4 [Stopwords elimination](#3.4-Stopwords-elimination)

## 1. Data loading

In [1]:
import json
from pprint import pprint

IN_PATH = '../data/squad/'


def load_data(filename):
    data = []
    with open(filename) as f:
        data = json.load(f)  
        
    return data


train = load_data(IN_PATH + 'train-v1.1.json')
dev = load_data(IN_PATH + 'dev-v1.1.json')

## 2. Structure of the data 
** The structure of the data can be seen in the following way:  **
<pre>       
            dataset   
            (dict)   
            /    \  
           /      \  
      'version'   'data'
      (string)    (list)
                  # each element in the list represents an article (dict)  
                     |
                     |
                  articles
                   (dict)
                   /    \
                  /      \
             'title'   'paragraphs'  
             (string)     (list)
                          # each element in the list is an dict with 'qas' and 'context' keys
                             |
                             |
                            pair
                           (dict)
                          /      \
                         /        \
                        /          \
                       /            \
                 'context'          'qas'
                 (string)           (list)
                  # context of      # each element in the list is a dict with 3 keys
                    the questions      |
                    and answers        |
                                       |
                                 question-answer
                                     (dict)
                                    /   |   \
                                   /    |    \
                                  /     |     \
                               'id' 'question' 'answers'
                             (string)(string)   (list)
                                                # each element in the list is an answer dict with 
                                                'text' of the answer and 'answer_start'
                                                  |
                                                  |
                                                answer
                                                (dict)
                                                /     \
                                               /       \
                                        'answer_start' 'text'
                                            (int)     (string)                                   
</pre>

## 3. Data preprocessing

### 3.1 Context/passage sentence tokenization [[1]](#References)
** Let's split sentences for all passages. These sentences are candidate answers for the questions from the same paragraph. **

In [2]:
from nltk.tokenize import sent_tokenize


def tokenize_sent_from_context(dataset):
    for article in dataset['data']:
        for qas_context in article['paragraphs']:
            qas_context['context_sentences'] = sent_tokenize(qas_context['context'])

In [3]:
tokenize_sent_from_context(dev)
pprint(dev['data'][0]['paragraphs'][0]['context_sentences'])

[u'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season.',
 u'The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24\u201310 to earn their third Super Bowl title.',
 u"The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California.",
 u'As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.']


### 3.2 Word tokenization [[1]](#References)
** Word tokenization for questions and context with removal of non-alphanumeric tokens.**

In [4]:
from nltk.tokenize import WordPunctTokenizer


def remove_non_alnum(word_list):
    return [word.lower() for word in word_list if word.isalnum()]


def tokenize_words(dataset):
    word_punct_tokenizer = WordPunctTokenizer()
    for article in dataset['data']:
        for qas_context in article['paragraphs']:
            # tokenize all context_sentences
            qas_context['context_sentences_words'] = list()
            for sentence in qas_context['context_sentences']:
                word_list = word_punct_tokenizer.tokenize(sentence)
                qas_context['context_sentences_words'].append(remove_non_alnum(word_list))
            
            # tokenize questions
            for qas in qas_context['qas']:
                question = qas['question']
                word_list = word_punct_tokenizer.tokenize(question)
                qas['question_words'] = remove_non_alnum(word_list)

In [5]:
tokenize_words(dev)

In [6]:
pprint(dev['data'][0]['paragraphs'][0]['context_sentences'][0])
pprint(dev['data'][0]['paragraphs'][0]['context_sentences_words'][0])

u'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season.'
[u'super',
 u'bowl',
 u'50',
 u'was',
 u'an',
 u'american',
 u'football',
 u'game',
 u'to',
 u'determine',
 u'the',
 u'champion',
 u'of',
 u'the',
 u'national',
 u'football',
 u'league',
 u'nfl',
 u'for',
 u'the',
 u'2015',
 u'season']


In [7]:
pprint(dev['data'][0]['paragraphs'][0]['qas'][0]['question'])
pprint(dev['data'][0]['paragraphs'][0]['qas'][0]['question_words'])

u'Which NFL team represented the AFC at Super Bowl 50?'
[u'which',
 u'nfl',
 u'team',
 u'represented',
 u'the',
 u'afc',
 u'at',
 u'super',
 u'bowl',
 u'50']


### 3.3 Lemmatization [[2]](2)

In [8]:
from nltk.stem import WordNetLemmatizer


lemmatizer = WordNetLemmatizer()
def lemmatize_tokens(tokens):
    lemmas = [lemmatizer.lemmatize(token, pos='v') for token in tokens]
    return lemmas


def lemmatization(dataset):
    word_punct_tokenizer = WordPunctTokenizer()
    for article in dataset['data']:
        for qas_context in article['paragraphs']:
            # lemmatize all context_sentences_words
            qas_context['context_sentences_lemmas'] = list()
            for word_list in qas_context['context_sentences_words']:
                qas_context['context_sentences_lemmas'].append(lemmatize_tokens(word_list))
            
            # lemmatize questions
            for qas in qas_context['qas']:
                word_list = qas['question_words']
                qas['question_lemmas'] = lemmatize_tokens(word_list)

In [9]:
lemmatization(dev)

In [10]:
pprint(dev['data'][0]['paragraphs'][0]['context_sentences'][2])
pprint(dev['data'][0]['paragraphs'][0]['context_sentences_words'][2])
pprint(dev['data'][0]['paragraphs'][0]['context_sentences_lemmas'][2])

u"The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California."
[u'the',
 u'game',
 u'was',
 u'played',
 u'on',
 u'february',
 u'7',
 u'2016',
 u'at',
 u'levi',
 u's',
 u'stadium',
 u'in',
 u'the',
 u'san',
 u'francisco',
 u'bay',
 u'area',
 u'at',
 u'santa',
 u'clara',
 u'california']
[u'the',
 u'game',
 u'be',
 u'play',
 u'on',
 u'february',
 u'7',
 u'2016',
 u'at',
 u'levi',
 u's',
 u'stadium',
 u'in',
 u'the',
 u'san',
 u'francisco',
 u'bay',
 u'area',
 u'at',
 u'santa',
 u'clara',
 u'california']


In [11]:
pprint(dev['data'][0]['paragraphs'][0]['qas'][0]['question'])
pprint(dev['data'][0]['paragraphs'][0]['qas'][0]['question_words'])
pprint(dev['data'][0]['paragraphs'][0]['qas'][0]['question_lemmas'])

u'Which NFL team represented the AFC at Super Bowl 50?'
[u'which',
 u'nfl',
 u'team',
 u'represented',
 u'the',
 u'afc',
 u'at',
 u'super',
 u'bowl',
 u'50']
[u'which',
 u'nfl',
 u'team',
 u'represent',
 u'the',
 u'afc',
 u'at',
 u'super',
 u'bowl',
 u'50']


### 3.4 Stopwords elimination

In [12]:
from nltk.corpus import stopwords


def remove_stopwords(tokens):
    stopwords_set = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in set(stopwords_set)]   
    return filtered_tokens


def stopwords_elimination(dataset):
    for article in dataset['data']:
        for qas_context in article['paragraphs']:
            # remove stopwords from context_sentences_lemmas
            qas_context['context_sentences_lemmas_without_stopwords'] = list()
            for lemmas_list in qas_context['context_sentences_lemmas']:
                qas_context['context_sentences_lemmas_without_stopwords'].append(remove_stopwords(lemmas_list))
            
            # remove stopwords from questions lemmas
            for qas in qas_context['qas']:
                lemmas_list = qas['question_lemmas']
                qas['question_lemmas_without_stopwords'] = remove_stopwords(lemmas_list)

In [13]:
stopwords_elimination(dev)

In [14]:
pprint(dev['data'][0]['paragraphs'][0]['context_sentences'][2])
pprint(dev['data'][0]['paragraphs'][0]['context_sentences_words'][2])
pprint(dev['data'][0]['paragraphs'][0]['context_sentences_lemmas'][2])
pprint(dev['data'][0]['paragraphs'][0]['context_sentences_lemmas_without_stopwords'][2])

u"The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California."
[u'the',
 u'game',
 u'was',
 u'played',
 u'on',
 u'february',
 u'7',
 u'2016',
 u'at',
 u'levi',
 u's',
 u'stadium',
 u'in',
 u'the',
 u'san',
 u'francisco',
 u'bay',
 u'area',
 u'at',
 u'santa',
 u'clara',
 u'california']
[u'the',
 u'game',
 u'be',
 u'play',
 u'on',
 u'february',
 u'7',
 u'2016',
 u'at',
 u'levi',
 u's',
 u'stadium',
 u'in',
 u'the',
 u'san',
 u'francisco',
 u'bay',
 u'area',
 u'at',
 u'santa',
 u'clara',
 u'california']
[u'game',
 u'play',
 u'february',
 u'7',
 u'2016',
 u'levi',
 u'stadium',
 u'san',
 u'francisco',
 u'bay',
 u'area',
 u'santa',
 u'clara',
 u'california']


In [15]:
pprint(dev['data'][0]['paragraphs'][0]['qas'][0]['question'])
pprint(dev['data'][0]['paragraphs'][0]['qas'][0]['question_words'])
pprint(dev['data'][0]['paragraphs'][0]['qas'][0]['question_lemmas'])
pprint(dev['data'][0]['paragraphs'][0]['qas'][0]['question_lemmas_without_stopwords'])

u'Which NFL team represented the AFC at Super Bowl 50?'
[u'which',
 u'nfl',
 u'team',
 u'represented',
 u'the',
 u'afc',
 u'at',
 u'super',
 u'bowl',
 u'50']
[u'which',
 u'nfl',
 u'team',
 u'represent',
 u'the',
 u'afc',
 u'at',
 u'super',
 u'bowl',
 u'50']
[u'nfl', u'team', u'represent', u'afc', u'super', u'bowl', u'50']


### References
[1] http://textminingonline.com/dive-into-nltk-part-ii-sentence-tokenize-and-word-tokenize  
[2] http://textminingonline.com/dive-into-nltk-part-iv-stemming-and-lemmatization